# InferenceData Cookbook

Here we present a collection of common manipulations when working with InferenceData

In [2]:
import arviz as az

In [3]:
idata = az.load_arviz_data("centered_eight")
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In [4]:
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

## Combine chains and draws

In [5]:
stacked = idata.posterior.stack(draws=("chain", "draw"))
stacked

<xarray.Dataset>
Dimensions:  (draws: 2000, school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
  * draws    (draws) MultiIndex
  - chain    (draws) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
  - draw     (draws) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
Data variables:
    mu       (draws) float64 -3.477 -2.456 -2.826 -1.996 ... 4.597 5.899 0.1614
    theta    (school, draws) float64 1.669 -6.239 2.195 ... -1.095 4.013 4.523
    tau      (draws) float64 3.73 2.075 3.703 4.146 ... 8.589 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

## Obtain a numpy array for a given parameter

Let's say we want to get the values for `mu` as a numpy array 

In [6]:
stacked.mu.values

array([-3.47698606, -2.45587061, -2.82625433, ...,  4.59705819,
        5.89850592,  0.16138927])

## Get the number of variables

How many groups are in our hierarchical model?

In [7]:
len(idata.observed_data.school)

8

## Getting the variable names

Which are the name for the groups in our hierarchical model?

In [8]:
idata.observed_data.school.values

array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
       'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'],
      dtype=object)

## Get a subset of chains

Let say we want to evaluate only chain 0 and 2 

In [9]:
idata.sel(chain=[0, 2]).posterior

<xarray.Dataset>
Dimensions:  (chain: 2, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 2
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... -1.571 -4.435 9.763
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 12.01 16.67
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 2.812 12.18 4.453
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

## Remove the first n draws (burn-in)

Let say that we have reasons to remove the first 100 samples, from all InferenceData groups with draws and from all chains

In [10]:
burnin = idata.sel(draw=slice(100, None))

If you check the `burnin` object you will see that the groups `posterior`, `posterior_predictive`, `prior` and `sample_stats` have 400 draws comapred to idata that has 500. The group `observed_data` has not been affected as this group does not have the `draw` dimension. ALternativelly you can specify wich group or groups you want to change.

In [11]:
burnin_posterior = idata.sel(draw=slice(100, None), groups="posterior")

## Compute posterior mean values along draw and chains dimensions

Let say you want to compute the mean value of the posterior samples, you can simply do


In [12]:
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

This will efectivelly compute the mean along all dimension. This is probably what you want for `mu` and `tau`,which have two dimensions (`chain` and `draw`), but maybe not what you expected for theta, which has one more dimension `school`. You can specify along wich dimension you want to compute the mean (or other functions).

In [13]:
idata.posterior.mean(dim=['chain', 'draw'])

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 4.093
    theta    (school) float64 6.026 4.724 3.576 4.478 3.064 3.821 6.25 4.544
    tau      float64 4.089